## Welcome to Agentic AI - Day 3 foundations

In [ ]:
# If you don't know what any of these package do - you can ask chatgpt as a Guide.

from dotenv import load_dotenv
from langchain_experimental.graph_transformers.llm import system_prompt
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr


In [ ]:
load_dotenv(override=True)
openai = OpenAI()

In [ ]:
reader = PdfReader("data/resume.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [ ]:
print(linkedin)

In [ ]:
with open('data/summary.text', "r", encoding="utf-8") as f:
    summary = f.read()

In [ ]:
name = "Nikhil Arora"

In [ ]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and linkedin profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn profile:\n{linkedin}\n\n"
system_prompt += f"with this context, please chat with the user, always staing in character as {name}."

In [ ]:
system_prompt

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
# create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [ ]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a user and an Agent. Your Task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is Playing the role of {name} and uts representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if it taking to a potential client or future employer who came across the website. \
THe agent has been provided with context on {name} in the form of their summary and Linkedin Details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With the context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [ ]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the user: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [ ]:
import os
gemini = OpenAI(api_key=os.getenv("GOOGLE_API_KEY"),
                base_url="https://generativelangague.googleapis.com/1vbeta/openai/")

In [ ]:
def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [ ]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-5-mini", messages=messages)
reply = response.choices.message.content

In [ ]:
reply

In [ ]:
evaluate(reply, "do you hold a patent?", messages[:1])

In [ ]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\n you just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer: \n{reply}\n\n"
    updated_system_prompt += f" Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{{"role": "user", "content": message}}]
    response = openai.chat.completions.create(model="gpt-5-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\n Everything in your reply needs to be in pig latin - \
            it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{{"role": "user", "content": message}}]
    response = openai.chat.completions.create(model="gpt-5-mini", messages=messages)
    reply = response.choices[0].message.content

    evaluation = evaluate(reply, message, history)

    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()